אוראל חגי 314831819
דוד תורגמן 314735564 

https://github.com/aaddii314/apartments_project

In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [90]:
import requests
from bs4 import BeautifulSoup

In [368]:
import pandas as pd
from urllib.parse import quote

In [705]:
url1 = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17544" #נאות אפקה א
url2 = "https://www.ad.co.il/nadlanrent?sp275=17413&sp276=17414&sp277=18236" #פארק צמרת
url3 = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=18511" #קריה
all_data=[]
all_data.append(get_apartment_details(url1))
all_data.append(get_apartment_details(url2))
all_data.append(get_apartment_details(url3))
big_df = pd.concat(all_data, ignore_index=True)


5
14
38
jbjbjbj
jbjbjbj
jbjbjbj
jbjbjbj
jbjbjbj
jbjbjbj


In [707]:
new_df = big_df.rename(columns={
    'חניה': 'has_parking',
    'מחסן': 'has_storage',
    'מעלית': 'elevator',
    'מיזוג': 'ac',
    'נגישות': 'handicap',
    'סורגים': 'has_bars',
    'ממ"ד': 'has_safe_room',
    'מרפסת': 'has_balcony',
    'מרוהטת': 'is_furnished',
    'משופצת': 'is_renovated',
    'מחיר': 'price',
    'מזגן': 'ac'
})
csv_data=new_df.to_csv('apartments.csv', index=False, encoding='utf-8-sig')
num_columns = len(big_df.columns)
print(num_columns)
print (new_df.columns)

26
Index(['property_type', 'neighborhood', 'address', 'room_num', 'floor',
       'total_floors', 'area', 'days_to_enter', 'num_of_payments',
       'monthly_arnona', 'building_tax', 'description', 'is_furnished',
       'has_bars', 'ac', 'has_parking', 'has_safe_room', 'has_balcony',
       'handicap', 'elevator', 'has_storage', 'is_renovated', 'price',
       'num_of_images', 'distance_from_center', 'garden_area'],
      dtype='object')


In [648]:
#def result1(adress_url)
    #result=get_apartment_details(adress_url)

In [713]:
def distance (accept_address):
    url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17544"
    key="************************"
    address="דיזינגוף סנטר,תל אביב"
    origin_encoded = quote(address)
    destination_encoded = quote(accept_address)
    ser=f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin_encoded}&destinations={destination_encoded}&key={key}&language=he"
    #response1 = requests.get(ser)
    #data = response1.json()
    
    
    response = requests.get(ser)
    if not response.status_code == 200:
        print("HTTP error",response.status_code)
        return "nun"
    else:

        response_data = response.json()
        distance_meters = float(response_data['rows'][0]['elements'][0]['distance']['value'])
        return distance_meters 
                    



In [703]:
url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17544"
def get_apartment_details(url_apartment):
    response = requests.get(url_apartment)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ads = soup.find_all('div', class_='card-block', attrs={'data-id': True})
        print(len(ads))
        
        apartment=[]
        for i in ads:
            links="https://www.ad.co.il"+( i.find('a').get('href'))
            response1 = requests.get(links)
            #print(links)
            soup1 = BeautifulSoup(response1.content, 'html.parser')
            price = soup1.find("table", class_="table table-sm mb-4")
            rows = price.find_all('tr')
            dicty={}

            for row in rows:
                dave=row.find_all('td')
                #for dav in dave: 
                    #if "דירה" in dav.text:
                if dave[0].text.strip()=="פרטי הנכס":
                    dicty["property_type"]=str(dave[1].text.strip())
                elif dave[0].text.strip()=="שכונה":
                    dicty["neighborhood"]=str(dave[1].text.strip())
                elif dave[0].text.strip()=="כתובת":
                    dicty["address"]=str(dave[1].text.strip())
                    place=dave[1].text.strip()+ " " + "תל אביב"
                elif dave[0].text.strip()=="חדרים":
                    dicty["room_num"]=float(dave[1].text.strip())
                elif dave[0].text.strip()=="קומה":
                    floar_info=dave[1].text.strip()
                    parts = floar_info.split("מתוך")
                    try:
                        dicty["floor"] = int(parts[0].strip())
                    except:
                        dicty["floor"]=0
                    try:
                        dicty["total_floors"] = int(parts[1].strip())
                    except:
                        dicty["total_floors"]=0
                elif dave[0].text.strip()=="שטח בנוי":
                    dicty["area"]=int(dave[1].text.strip())
                elif dave[0].text.strip()=="שטח גינה":
                    dicty["garden_area"]=int(dave[1].text.strip())   
                elif dave[0].text.strip()=="תאריך כניסה":
                    value = dave[1].text.strip()
                    try:
                        dicty["days_to_enter"] = int(value)
                    except:
                        dicty["days_to_enter"] = 0 
                elif dave[0].text.strip()=="תשלומים בשנה":
                    dicty["num_of_payments"]=int(dave[1].text.strip())
                elif dave[0].text.strip()=="ארנונה בחודש":
                    dicty["monthly_arnona"]=int(dave[1].text.strip())
                elif dave[0].text.strip()=="ועד בית בחודש":
                    dicty["building_tax"]=int(dave[1].text.strip())
                #elif dave[0].text.strip()=="חדרים":
                    #dicty["total_floors"]=dave[1].text.strip()[2]
            desc=soup1.find("p", class_="text-word-break")
            dicty["description"]=str(desc.text.replace("\n", " ").strip())
            more_desc=soup1.find_all("div",class_="card-icon col-6 d-inline disabled")
            for i in more_desc:
                #if i.find("i",class_="fas fa-fw fa-check"):
                span=i.find("span")
                dicty[span.text.strip()]=0
            more_desc1=soup1.find_all("div",class_="card-icon col-6 d-inline")
            for i in more_desc1:
                span=i.find("span")
                dicty[span.text.strip()]=1
            del dicty['מרפסת שמש']
            del dicty ['על עמודים']
            price= soup1.find("div", class_="d-flex justify-content-between").find_all("h2", class_="card-title")[1].text.strip()
            clean_price = price.replace("₪", "").replace(",", "").strip()
            dicty["price"] = float(clean_price)
            gallery = soup1.find_all("a", itemprop="contentUrl")
            num_of_images = int(len(gallery))
            dicty['num_of_images'] = num_of_images
            try:
                distance_dizingof=distance(place)
                dicty["distance_from_center"]=distance_dizingof
            except:
                dicty["distance_from_center"]="nune"
            apartment.append(dicty)
        df = pd.DataFrame(apartment)
        return (df)
                
                        
                    
                    
    
    
                
                
                        
                        
                    
    
    
    
    
    
             
